In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

ModuleNotFoundError: No module named 'lab_black'

In [1]:
import numpy as np
import pandas as pd
import torch
import sys

from sklearn.manifold import TSNE
import plotly.graph_objects as go

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/Users/Zahed001/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Model and Vocabulary

In [2]:
folder = "../weights/cbow_histaware_test_67"

model = torch.load(f"../{folder}/model.pt", map_location=device)
vocab = torch.load(f"../{folder}/vocab.pt")

ModuleNotFoundError: No module named 'utils'

## Getting Embeddings

In [33]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape

(15381, 300)

# Visualization with t-SNE

In [9]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
embeddings_df_trans.index = vocab.get_itos()

# if token is a number
is_numeric = embeddings_df_trans.index.str.isnumeric()

/Users/Zahed001/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/Zahed001/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [10]:
color = np.where(is_numeric, "green", "black")
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color),
    )
)
fig.write_html("../word2vec_visualization.html")

# Find Similar Words

In [34]:
def get_top_similar(word: str, topN: int = 10):
    word_id = vocab[word]
    if word_id == 0:
        print("Out of vocabulary word")
        return

    word_vec = embeddings_norm[word_id]
    word_vec = np.reshape(word_vec, (len(word_vec), 1))
    dists = np.matmul(embeddings_norm, word_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_word_id in topN_ids:
        sim_word = vocab.lookup_token(sim_word_id)
        topN_dict[sim_word] = dists[sim_word_id]
    return topN_dict

In [35]:
for word, sim in get_top_similar("gas").items():
    print("{}: {:.3f}".format(word, sim))

aardgas: 0.594
water: 0.446
beladen: 0.435
vuur: 0.423
afgewerkte: 0.385
tweetal: 0.372
seconde: 0.355
koelwater: 0.354
groente: 0.353
wegdek: 0.343


In [37]:
for word, sim in get_top_similar("aardolie").items():
    print("{}: {:.3f}".format(word, sim))

olie: 0.463
boort: 0.452
delfstoffen: 0.431
Groot-Brittannië: 0.422
nikkel: 0.422
elektriciteit: 0.419
proefboringen: 0.417
Brunei: 0.397
Alaska: 0.386
concessiegebied: 0.386


In [36]:
for word, sim in get_top_similar("steenkool").items():
    print("{}: {:.3f}".format(word, sim))

vloeibaar: 0.368
energiebronnen: 0.356
ammoniak: 0.354
elektriciteit: 0.344
steenkolen: 0.337
stookolie: 0.323
kolen: 0.323
grotendeels: 0.321
grondstof: 0.320
aardolie: 0.319


# Vector Equations

In [ ]:
emb1 = embeddings[vocab["king"]]
emb2 = embeddings[vocab["man"]]
emb3 = embeddings[vocab["woman"]]

emb4 = emb1 - emb2 + emb3
emb4_norm = (emb4 ** 2).sum() ** (1 / 2)
emb4 = emb4 / emb4_norm

emb4 = np.reshape(emb4, (len(emb4), 1))
dists = np.matmul(embeddings_norm, emb4).flatten()

top5 = np.argsort(-dists)[:5]

for word_id in top5:
    print("{}: {:.3f}".format(vocab.lookup_token(word_id), dists[word_id]))